# Lab 12: Stored Procedures and Views - Interactive Practice

This notebook provides hands-on practice with MySQL stored procedures, views, and indexes. We'll connect to a MySQL database and execute various procedures and queries to understand these advanced database concepts.

## Learning Objectives
- Create and execute stored procedures with parameters
- Implement error handling in stored procedures
- Work with transactions in stored procedures
- Create and query database views
- Understand index optimization
- Combine procedures and views for complex operations

## Prerequisites
- MySQL Server running
- mysql-connector-python installed
- Database and tables from examples.sql created

## Setup

In [ ]:
# Install required packages if not already installed
# !pip install mysql-connector-python

# Import required libraries
import mysql.connector
from mysql.connector import Error
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Database connection configuration
config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'your_password',  # Replace with your MySQL password
    'database': 'procedures_views_db',
    'autocommit': False  # We'll manage transactions manually
}

def create_connection():
    """Create database connection"""
    try:
        connection = mysql.connector.connect(**config)
        print("✅ Connected to MySQL database")
        return connection
    except Error as e:
        print(f"❌ Error connecting to MySQL: {e}")
        return None

# Test connection
conn = create_connection()
if conn:
    conn.close()
    print("✅ Connection test successful")
else:
    print("❌ Please check your database configuration")

## Exercise 1: Basic Stored Procedures

Let's start by calling some basic stored procedures to understand how they work.

In [ ]:
def execute_query(connection, query, description=""):
    """Execute a SELECT query and return results as DataFrame"""
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        
        # Get column names
        columns = [desc[0] for desc in cursor.description] if cursor.description else []
        
        # Get all rows
        rows = cursor.fetchall()
        
        cursor.close()
        
        if description:
            print(f"\n{description}")
        
        if rows:
            df = pd.DataFrame(rows, columns=columns)
            return df
        else:
            print("No results returned")
            return None
            
    except Error as e:
        print(f"❌ Error executing query: {e}")
        return None

def call_procedure(connection, procedure_name, params=None, description=""):
    """Call a stored procedure and return results"""
    try:
        cursor = connection.cursor()
        
        if params:
            cursor.callproc(procedure_name, params)
        else:
            cursor.callproc(procedure_name)
        
        # Get results from all result sets
        results = []
        for result in cursor.stored_results():
            columns = [desc[0] for desc in result.description]
            rows = result.fetchall()
            if rows:
                results.append(pd.DataFrame(rows, columns=columns))
        
        cursor.close()
        
        if description:
            print(f"\n{description}")
        
        return results if results else None
        
    except Error as e:
        print(f"❌ Error calling procedure {procedure_name}: {e}")
        return None

# Connect to database
conn = create_connection()

if conn:
    # Exercise 1.1: Call GetAllEmployees procedure
    results = call_procedure(conn, 'GetAllEmployees', 
                           description="📋 Exercise 1.1: Get all active employees")
    if results:
        for df in results:
            display(df)
    
    # Exercise 1.2: Call GetEmployeeCount procedure
    results = call_procedure(conn, 'GetEmployeeCount', 
                           description="📊 Exercise 1.2: Get employee count")
    if results:
        for df in results:
            display(df)
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 2: Stored Procedures with Parameters

Now let's practice calling procedures with input parameters.

In [ ]:
# Connect to database
conn = create_connection()

if conn:
    # Exercise 2.1: Get employees by department
    departments = ['IT', 'Sales', 'HR']
    
    for dept in departments:
        results = call_procedure(conn, 'GetEmployeesByDepartment', [dept], 
                               description=f"👥 Exercise 2.1: Employees in {dept} department")
        if results:
            for df in results:
                display(df.head())  # Show first few results
        print("---")
    
    # Exercise 2.2: Get employees by salary range
    salary_ranges = [
        (50000, 70000),
        (70000, 90000)
    ]
    
    for min_sal, max_sal in salary_ranges:
        results = call_procedure(conn, 'GetEmployeesBySalaryRange', [min_sal, max_sal], 
                               description=f"💰 Exercise 2.2: Employees with salary ${min_sal:,} - ${max_sal:,}")
        if results:
            for df in results:
                display(df)
        print("---")
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 3: Procedures with OUT Parameters

Let's work with procedures that return values through OUT parameters.

In [ ]:
def call_procedure_with_out_params(connection, procedure_name, in_params, out_param_count, description=""):
    """Call procedure with OUT parameters and return their values"""
    try:
        cursor = connection.cursor()
        
        # Prepare parameters: IN params + OUT placeholders
        params = list(in_params) + [0] * out_param_count
        
        # Call procedure
        cursor.callproc(procedure_name, params)
        
        # Get OUT parameter values
        cursor.execute("SELECT " + ", ".join([f"@{procedure_name}_{i}" for i in range(out_param_count)]))
        out_values = cursor.fetchone()
        
        cursor.close()
        
        if description:
            print(f"\n{description}")
        
        return out_values
        
    except Error as e:
        print(f"❌ Error calling procedure {procedure_name}: {e}")
        return None

# Connect to database
conn = create_connection()

if conn:
    # Exercise 3.1: Get department statistics
    departments = ['IT', 'Sales', 'Finance']
    
    print("📈 Exercise 3.1: Department Statistics")
    for dept in departments:
        out_values = call_procedure_with_out_params(
            conn, 'GetDepartmentStats', [dept], 3,
            description=f"Statistics for {dept} department:"
        )
        
        if out_values:
            emp_count, avg_salary, total_budget = out_values
            print(f"  Employees: {emp_count}")
            print(f"  Average Salary: ${avg_salary:,.2f}")
            print(f"  Department Budget: ${total_budget:,.2f}")
        print("---")
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 4: Error Handling and Transactions

Let's test procedures with error handling and transaction management.

In [ ]:
# Connect to database
conn = create_connection()

if conn:
    # Exercise 4.1: Test SafeEmployeeInsert with valid data
    print("✅ Exercise 4.1: Testing SafeEmployeeInsert with valid data")
    
    test_employees = [
        ('Test Employee 1', 'IT', 75000.00, 'test1@company.com'),
        ('Test Employee 2', 'HR', 65000.00, 'test2@company.com')
    ]
    
    for name, dept, salary, email in test_employees:
        try:
            cursor = conn.cursor()
            cursor.callproc('SafeEmployeeInsert', [name, dept, salary, email])
            
            # Get result
            for result in cursor.stored_results():
                df = pd.DataFrame(result.fetchall(), columns=[desc[0] for desc in result.description])
                display(df)
            
            cursor.close()
            conn.commit()
            
        except Error as e:
            print(f"❌ Error inserting {name}: {e}")
            conn.rollback()
    
    # Exercise 4.2: Test error handling with invalid data
    print("\n❌ Exercise 4.2: Testing error handling with invalid data")
    
    invalid_employees = [
        ('', 'IT', 75000.00, 'invalid1@company.com'),  # Empty name
        ('Invalid Employee', 'IT', 25000.00, 'invalid2@company.com'),  # Low salary
        ('Invalid Employee 2', 'InvalidDept', 65000.00, 'invalid3@company.com')  # Invalid department
    ]
    
    for name, dept, salary, email in invalid_employees:
        try:
            cursor = conn.cursor()
            cursor.callproc('SafeEmployeeInsert', [name, dept, salary, email])
            
            for result in cursor.stored_results():
                df = pd.DataFrame(result.fetchall(), columns=[desc[0] for desc in result.description])
                display(df)
            
            cursor.close()
            conn.commit()
            
        except Error as e:
            print(f"Expected error for {name}: {e}")
            conn.rollback()
    
    # Exercise 4.3: Test TransferEmployee procedure
    print("\n🔄 Exercise 4.3: Testing employee transfer")
    
    # Get an employee ID first
    df = execute_query(conn, "SELECT id, name, department FROM employees WHERE active = TRUE LIMIT 1")
    if df is not None and not df.empty:
        emp_id = df.iloc[0]['id']
        emp_name = df.iloc[0]['name']
        
        print(f"Transferring {emp_name} (ID: {emp_id}) to Finance department")
        
        results = call_procedure(conn, 'TransferEmployee', [emp_id, 'Finance'])
        if results:
            for df in results:
                display(df)
        
        # Verify the transfer
        df_after = execute_query(conn, f"SELECT name, department FROM employees WHERE id = {emp_id}")
        if df_after is not None:
            display(df_after)
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 5: Working with Views

Now let's explore database views and how to query them.

In [ ]:
# Connect to database
conn = create_connection()

if conn:
    # Exercise 5.1: Query active_employees view
    df = execute_query(conn, "SELECT * FROM active_employees ORDER BY department, name",
                      description="👥 Exercise 5.1: Active employees view")
    if df is not None:
        display(df)
    
    # Exercise 5.2: Query employee salary analysis view
    df = execute_query(conn, "SELECT * FROM employee_salary_analysis ORDER BY avg_salary DESC",
                      description="📊 Exercise 5.2: Employee salary analysis")
    if df is not None:
        display(df)
    
    # Exercise 5.3: Query project details view
    df = execute_query(conn, "SELECT * FROM project_details ORDER BY budget_percentage DESC LIMIT 5",
                      description="📋 Exercise 5.3: Top 5 projects by budget percentage")
    if df is not None:
        display(df)
    
    # Exercise 5.4: Query department performance view
    df = execute_query(conn, "SELECT * FROM dept_performance ORDER BY budget_utilization DESC",
                      description="🏆 Exercise 5.4: Department performance ranking")
    if df is not None:
        display(df)
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 6: Advanced Procedures and Views

Let's work with more complex procedures that combine multiple concepts.

In [ ]:
# Connect to database
conn = create_connection()

if conn:
    # Exercise 6.1: Generate department reports
    departments = ['IT', 'Sales']
    
    for dept in departments:
        print(f"\n📊 Exercise 6.1: Comprehensive report for {dept} department")
        
        results = call_procedure(conn, 'GenerateDepartmentReport', [dept])
        if results:
            for i, df in enumerate(results):
                if not df.empty:
                    display(df)
                    print("---")
        else:
            print(f"No data available for {dept} department")
    
    # Exercise 6.2: Dynamic employee search
    print("\n🔍 Exercise 6.2: Dynamic employee search")
    
    search_criteria = [
        ('name', 'Aa'),
        ('department', 'IT'),
        ('email', 'company.com')
    ]
    
    for column, value in search_criteria:
        results = call_procedure(conn, 'DynamicEmployeeSearch', [column, value],
                               description=f"Searching {column} for '{value}'")
        if results:
            for df in results:
                display(df)
        print("---")
    
    # Exercise 6.3: Process salary increase
    print("\n💰 Exercise 6.3: Processing salary increase")
    
    # Apply 3% increase to Marketing department
    results = call_procedure(conn, 'ProcessSalaryIncrease', ['Marketing', 3.0])
    if results:
        for df in results:
            display(df)
    
    # Verify the changes
    df = execute_query(conn, "SELECT name, department, salary FROM employees WHERE department = 'Marketing'",
                      description="Marketing department salaries after 3% increase:")
    if df is not None:
        display(df)
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 7: Index Performance Analysis

Let's analyze how indexes affect query performance.

In [ ]:
def analyze_query_performance(connection, query, description=""):
    """Analyze query performance using EXPLAIN"""
    try:
        cursor = connection.cursor()
        
        explain_query = f"EXPLAIN FORMAT=JSON {query}"
        cursor.execute(explain_query)
        
        result = cursor.fetchone()
        explain_json = result[0] if result else None
        
        cursor.close()
        
        if description:
            print(f"\n{description}")
        
        if explain_json:
            print("Query execution plan:")
            print(explain_json)
        
        return explain_json
        
    except Error as e:
        print(f"❌ Error analyzing query: {e}")
        return None

# Connect to database
conn = create_connection()

if conn:
    # Exercise 7.1: Analyze index usage
    queries = [
        ("SELECT name, salary FROM employees WHERE department = 'IT'", 
         "Query using department index"),
        ("SELECT name, email FROM employees WHERE email LIKE '%@company.com'", 
         "Query using email index"),
        ("SELECT name, salary FROM employees WHERE department = 'IT' AND salary > 70000", 
         "Query using composite index"),
        ("SELECT COUNT(*) FROM employees WHERE active = TRUE", 
         "Query on active column (no index)")
    ]
    
    for query, desc in queries:
        analyze_query_performance(conn, query, description=f"🔍 Exercise 7.1: {desc}")
    
    # Exercise 7.2: Show existing indexes
    df = execute_query(conn, "SHOW INDEXES FROM employees",
                      description="📋 Exercise 7.2: Current indexes on employees table")
    if df is not None:
        display(df[['Key_name', 'Column_name', 'Index_type']])
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 8: Business Intelligence Dashboard

Let's create a simple dashboard using our views and procedures.

In [ ]:
# Connect to database
conn = create_connection()

if conn:
    # Exercise 8.1: Company dashboard overview
    df = execute_query(conn, "SELECT * FROM company_dashboard",
                      description="📊 Exercise 8.1: Company Dashboard Overview")
    if df is not None:
        display(df)
    
    # Exercise 8.2: Monthly metrics for different periods
    months_to_check = [
        (2023, 6),
        (2023, 10)
    ]
    
    for year, month in months_to_check:
        print(f"\n📅 Exercise 8.2: Monthly metrics for {year}-{month:02d}")
        
        results = call_procedure(conn, 'GetMonthlyMetrics', [year, month])
        if results:
            for df in results:
                display(df)
    
    # Exercise 8.3: Audit log analysis
    df = execute_query(conn, 
                      "SELECT action, table_name, COUNT(*) as count "
                      "FROM audit_log "
                      "GROUP BY action, table_name "
                      "ORDER BY count DESC",
                      description="📝 Exercise 8.3: Audit log summary")
    if df is not None:
        display(df)
    
    # Exercise 8.4: Recent audit entries
    df = execute_query(conn, 
                      "SELECT action, table_name, record_id, user_name, action_date "
                      "FROM audit_log "
                      "ORDER BY action_date DESC LIMIT 10",
                      description="🕐 Exercise 8.4: Recent audit entries")
    if df is not None:
        display(df)
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 9: Advanced Practice

Try these advanced exercises to deepen your understanding.

In [ ]:
# Connect to database
conn = create_connection()

if conn:
    print("🎯 Exercise 9: Advanced Practice Challenges")
    print("\n1. Create a view that shows employee performance metrics")
    print("2. Write a procedure that generates quarterly reports")
    print("3. Analyze the impact of different indexes on query performance")
    print("4. Create a procedure that handles bulk employee updates")
    
    # Challenge 1: Employee performance view
    try:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE OR REPLACE VIEW employee_performance AS
            SELECT 
                e.id,
                e.name,
                e.department,
                e.salary,
                d.budget as dept_budget,
                ROUND(e.salary / d.budget * 100, 4) as salary_budget_ratio,
                COUNT(p.id) as project_count,
                ROUND(AVG(p.budget), 2) as avg_project_budget
            FROM employees e
            JOIN departments d ON e.department = d.name
            LEFT JOIN projects p ON d.id = p.department_id
            WHERE e.active = TRUE
            GROUP BY e.id, e.name, e.department, e.salary, d.budget
            ORDER BY salary_budget_ratio DESC;
        """)
        print("✅ Created employee_performance view")
        cursor.close()
        conn.commit()
        
        # Query the new view
        df = execute_query(conn, "SELECT * FROM employee_performance LIMIT 5",
                          description="Top 5 employees by salary-budget ratio:")
        if df is not None:
            display(df)
            
    except Error as e:
        print(f"❌ Error creating performance view: {e}")
        conn.rollback()
    
    # Challenge 2: Quarterly report procedure
    try:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE PROCEDURE GenerateQuarterlyReport(IN target_year INT, IN target_quarter INT)
            BEGIN
                DECLARE start_month INT;
                DECLARE end_month INT;
                
                SET start_month = (target_quarter - 1) * 3 + 1;
                SET end_month = target_quarter * 3;
                
                SELECT CONCAT('Q', target_quarter, ' ', target_year, ' Report') as report_title;
                
                SELECT 
                    'SUMMARY' as section,
                    COUNT(DISTINCT e.id) as total_employees,
                    COUNT(DISTINCT p.id) as total_projects,
                    ROUND(AVG(e.salary), 2) as avg_salary,
                    SUM(d.budget) as total_budget
                FROM employees e
                CROSS JOIN departments d
                CROSS JOIN projects p
                WHERE YEAR(e.hire_date) = target_year 
                  AND QUARTER(e.hire_date) = target_quarter;
            END
        """)
        print("✅ Created GenerateQuarterlyReport procedure")
        cursor.close()
        conn.commit()
        
        # Test the procedure
        results = call_procedure(conn, 'GenerateQuarterlyReport', [2023, 3],
                               description="Q3 2023 Quarterly Report:")
        if results:
            for df in results:
                display(df)
                
    except Error as e:
        print(f"❌ Error creating quarterly report procedure: {e}")
        conn.rollback()
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Summary

In this lab, we've covered:

1. **Basic Stored Procedures**: Creating and calling procedures without parameters
2. **Parameterized Procedures**: Working with IN and OUT parameters
3. **Error Handling**: Implementing robust error handling in procedures
4. **Transactions**: Managing database transactions in stored procedures
5. **Database Views**: Creating and querying complex views with JOINs
6. **Index Optimization**: Understanding how indexes improve query performance
7. **Advanced Procedures**: Dynamic SQL, cursors, and complex business logic
8. **Business Intelligence**: Creating dashboards and reports using views and procedures

### Key Takeaways:
- Stored procedures encapsulate business logic and improve performance
- Views simplify complex queries and provide data abstraction
- Proper indexing is crucial for query optimization
- Error handling and transactions ensure data integrity
- Combining procedures and views enables powerful database applications

### Next Steps:
1. Experiment with creating your own procedures and views
2. Analyze query performance with different indexing strategies
3. Implement more complex business logic in stored procedures
4. Create comprehensive reporting dashboards

Remember to always test your procedures and views thoroughly before deploying them in production environments!